In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut

out = JupyterOut.unit_square( )

# Functions and the Python Object Model

We may now consider the nature of functions as they operate within the unique model of object-oriented programming offered by Python. This section addresses three considerations: 
* Python functions are themselves first-class objects.
* Code contained within Python functions can access objects outside of their local frame.
* Functions direct the flow of program execution by spawning a cascade of sub-procedures called "the stack".

Earlier, we stated that while it is legal to refer to variables in a more global scope than the one currently executing, this practice is largely inadvisable. Below, we explore why this is the case. It is nearly always preferable to exchange data between a function and its calling context using the formal mechanisms of arguments and return values.




### Functions as First-Class Objects

A Python function is considered first-class because it enjoys all the
privileges of any other object in memory: 
* Functions may be assigned to variables. 
* Functions may be programmatically created, stored, and destroyed
* Functions may be offered as the return value of other functions. 
* Functions may be ***passed as an argument to other functions***.

The `map()` function is built into the Python language, and takes a function as one of its arguments. This function iteratively applies, or “maps”, a given function to a sequence of values that are regarded as arguments, and returns a collection of the resulting values. In its simplest form, it reproduces the process of looping over the given values and collecting the results.

The statement `map(func,vals)` is equivalent to the List comprehension 
    
    [func(v) for v in vals]

Multiple value Lists may be used.

The statement `map(func,vals_a,vals_b)` is exactly equivalent to the statement

    [func(a,b) for a,b in zip(vals_a,vals_b)]

The first piece of code below uses this function to limit a given collection of Points to a given x- and y-coordinate bounds. The second piece of code below defines a function that takes two arguments, and passes two sequences of values to the `map()` function. Both of these examples offer a new means to familiar ends.

In [ ]:

"""
Mapping a Function to A Sequence
Given a collection of Points pts, limits the coordinates of each Point
to a bounds defined by two Intervals by applying the built-in map 
function.
"""
ival_x = Interval(1,2)
ival_y = Interval(-1,1)

def limit_to_bnds(pt):
    pt.x = ival_x.limit_val(pt.x)
    pt.y = ival_y.limit_val(pt.y)
    return pt
    
pts = map(limit_to_bnds, pts)

In [ ]:
"""
Mapping a Function to Two Sequences
Given a collection of Segments segs, iteratively produces a new 
Segment that spans between a Point evaluated along each given 
Segment by applying the built-in map function.
"""
def func(seg_a, seg_b):
    return Segment(seg_a.eval(t),seg_b.eval(t))
    
for n in range(gens):
    # shift the List of Segments to make matching pairs
    shifted = [segs[-1]]+segs[:-1]
    segs = map(func, segs, shifted)


## Variable Scope and Lifetime

Variables initialized within a function are not available to code outside of that function, and are limited to the lifetime of the function. This is not true for variables defined within or as iterators for a loop, which do remain available outside the codeblock in which they are instantiated, and persist beyond the lifetime of the loop.

***What can account for this difference in the behavior?***

Two concepts in OOP offer a rationale:
* ***scope***, the parts of a program to which a variable is accessible
* ***lifetime***, the duration of the existence of a variable

Any variable defined in the main body of a script, including those within codeblocks related to control-flow structures, are called ***global variables***. These are available for access throughout the file (they have *global scope*) and persist for the duration of the shell (they have an *unlimited lifetime*) unless they are explicitly deleted. 

In contrast, any variable defined within the body of a function, including the
codeblocks contained within the function, are called ***local variables***. These are accessible only by subsequent statements within the function (they are *locally scoped*) and exist only as long as the function is executing (they have a *limited lifetime*). When the function is complete, these variables are removed from memory.

While they provide a connection to its calling context, ***the arguments of a function are treated as local variables***.

A series of small examples illustrates the distinction between local and global variables when working with functions.

### Scenario I
Here we demonstrate the typical situation of linking the interior and exterior of a function only by arguments. 

The variable `pt_a` is defined in the global scope before the initialization of the function `nudged()`, and remains available for the lifetime of the script. The variable `vec` is a local variable defined within the function, and persists only as the function is executing and is therefore unavailable outside of this scope.

In [ ]:
"""
Scenario I
The interior and exterior of a function are linked only through 
argument references
"""
pt_a = Point()
def nudged(pt,amt):
    vec = UZ * amt
    return pt + vec #{breakpoint a}
    
pt_b = nudged(pt_a,10)
pt_c = nudged(pt_b,20)  #{breakpoint b}

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D13.jpg" style="width:400px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D14.jpg" style="width:400px; display: inline;">

### Scenario II

Here we find a reference to a variable defined in a more global scope. Again, `pt_a` is a global variable and `vec` is a local variable, but in contrast to the code above, the global variable is referenced from within the body of the function. This is a perfectly legal reference to a more global scope, as `pt_a` is available at the point from which it is called at breakpoint C.

In [ ]:
"""
Scenario II
A variable defined in a more global scope is referenced from within 
a function
"""
pt_a = Point()
def nudged(amt):
    vec = UZ * amt
    return pt_a + vec  #{breakpoint c}
    
pt_b = nudged(10)
pt_c = nudged(20)



<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D15.jpg" style="width:400px; display: inline;">

### Scenario III

While access is permitted in these situations, assignment is not. 

Once again, `pt_a` is defined in the global scope and is available for access from within the function. However, when we attempt to assign to this variable within the function, creating a reference to a freshly initialized Point on the first line, something unexpected happens. 

Rather than re-assigning the existing global variable to a new object, an entirely new local one is created with the same name. Since this new local variable is available for access and modification within the function, the print statement at breakpoint D represents the values of this new Point at coordinates (0,0,0). 

It appears from within the function that we have modified `pt_a`, but in truth we have done so only with local effect. The global variable `pt_a` and its related object remain unaffected. At breakpoint E, after the function exits and the local variables are discarded, we see that the original value remains.

In [ ]:
"""
Scenario III
Does not work as intended. Assignment of a global variable from 
within a function results in the creation of a new local variable 
with the same name.
"""
pt_a = Point(0,0,10)
def reset():
    pt_a = Point()  #{breakpoint d}
    print pt_a, "at breakpoint D"

reset()
print pt_a, "at breakpoint E"  #{breakpoint e}



<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D16.jpg" style="width:400px; display: inline;">

### Scenario IV

Attempts to re-assign a variable that has been passed into a function as an argument. Since the arguments of a function are treated as local variables, this code misbehaves in much the same manner as the previous. 

While the global variable `pt_a` and the local argument `pt` both refer to the same object, at breakpoint F the object reference associated with `pt` is re-assigned to the result of the `pt + vec` expression (which produces an entirely new object), leaving the more global object reference intact. 

Again, any subsequent calls to `pt` within the scope of the function will reflect the values of the new object, but once the function exits, we can see that this is only a local effect as the global variable `pt_a` remains unaffected at breakpoint G.

In [ ]:

"""
Scenario IV
Does not work as intended. Re-assignment of an argument produces only 
local effect, and does not alter the variable reference in the more 
global scope.
"""
pt_a = Point()
def nudge(pt, amt):
    vec = UZ * amt
    pt = pt + vec  #{breakpoint f}
    print pt, "at breakpoint F"

nudge(pt_a, 10)
print pt_a, "at breakpoint G"  #{breakpoint g}

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D17.jpg" style="width:400px; display: inline;">

### Scenario V

To properly achieve the desired effect of these two scripts, and to re-assign a global variable from within a function, we must explicitly tell the Python shell that this is our intention. To do so requires the use of the keyword ***`global`***, which is used exclusively for this purpose.

In [ ]:
"""
Scenario V
The use of the global keyword allows for the assignment of a global 
variable
"""
pt_a = Point(0,0,10)
def reset():
    global pt_a
    pt_a = Point()  #{breakpoint h}
    print pt_a, "at breakpoint H"

reset()
print pt_a, "at breakpoint I"  #{breakpoint i}


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D18.jpg" style="width:400px; display: inline;">

### Scenario VI

All of the above examples operate on a global variable that refers to a structured object, but would manifest different behavior if operating on a primitive object. 

Given that we are operating on a structured object, and that out-of-scope access is treated differently than assignment, an alternative to using the `global` keyword is to avoid re-assigning the reference, and instead re-assigning each member. 

Consider the code below, which achieves the goal of “resetting” a Point to the world origin by individually setting its x,y, and z coordinates. Any similar modification of the members of an object referenced by a global variable is permitted without the `global` keyword.

In [ ]:
"""
Scenario VI
The global variable will register the effects of manipulating the 
members of structured objects within a function
"""
pt_a = Point(0,0,10)
def reset():
    pt_a.x, pt_a.y, pt_a.z = 0,0,0
    print pt_a, "at breakpoint J" #{breakpoint j}

reset()
print pt_a, "at breakpoint K" #{breakpoint k}

## The Stack

Functions involve a manipulation of the way in which the Python shell moves across our code from statement to statement, evaluating expressions and storing objects in memory. How are these movements organized across the multiplicity of frames engendered by function calls?

From our discussion of the evaluation of statements, We can imagine that the process of evaluating one set of statements might be “paused” when a function call is encountered, and must wait at that location until the results of another process that evaluates the contents of that function become available. 

This presumption is basically correct, and that the Python shell proceeds in this way through a mechanism called a ***call stack***.

The “stack”, is a collection-like data structure common to nearly all programming languages. Its purpose is to keep track of the ongoing processes and spaces of computer memory related to a program.

The main body of our script and its related frame as the first item on the stack. Each time a function is called, a new routine is spawned, and a new frame is added to the stack. This process continues, resulting in a cascade of frames.

Each time a function finishes executing, the related item on the stack is discarded and control returns to the context from which the function was called.

To illustrate, consider the movements of the interpreter and the nature of the stack as they navigate between the frames constructed by one of the subdivision routines presented earlier.

In [ ]:
for n in range(gens): 
    faces = subdivide()
    
"""
A function
"""
def subdivide():
    sfs = []
    while faces:
        fac = faces.pop()

        cen = Point.centroid([seg.ept for seg in fac])

        csubs = [Segment(seg.ept,cen) for seg in fac]
        sfs.append((fac[0],csubs[0],csubs[2].inverted()))
        sfs.append((fac[1],csubs[1],csubs[0].inverted()))
        sfs.append((fac[2],csubs[2],csubs[1].inverted()))
        
    return sfs

"""
The static method of the Point class (will not run in this context)

@staticmethod
def centroid(pts):
    return Point(Vec.average(pts))
    
"""

The main work of this routine is a simple loop across a series of numbered generations `gens`. We’ll call the frame associated with the main body of this code ***frame A***, as seen in the nearby control-flow diagram. 

At breakpoint A, we can see that a call to the `subdivide()` function has been reached, the results of which are assigned to the variable faces. At this point, the process of interpreting the code in ***frame A*** must wait, and a new process is spawned to interpret the called function and return a value.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.B01.jpg" style="width:800px; display: inline;">

This results in the creation of a new frame which we may call ***frame B***.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.B02.jpg" style="width:800px; display: inline;">

Here we see that that the process of interpreting ***frame A*** has been frozen in place (note the grayed-out breakpoint that marks the current position), while the process of interpreting the code related to ***frame B*** is carried out.

Any expression that requires another bit of code in its evaluation can be similarly unpacked. 

For example, the call to the method `Point.centroid()` at breakpoint B of ***frame B***. The code required to evaluate this expression is a part of the Decod.es library. While the lines of code required to produce a value are not present in our script, the process is exactly as if they were. 

At this point in the interpretation of the `subdivide()` function, the process is paused, and another is spawned and added to the stack. As we can see in the nearby control flow diagram, this new frame returns a value to its calling context after just one line of code.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.B03.jpg" style="width:800px; display: inline;">

After returning a value, ***frame C*** is discarded and all contained objects are deleted. Back in ***frame B***, with the value of the `Point.centroid()` method evaluated, the process of evaluating the `subdivide()` function may continue on its way.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.B04.jpg" style="width:800px; display: inline;">

Eventually, after some number of loops, the `return` statement is reached, and ***frame B*** meets the same fate as ***frame C***.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.B05.jpg" style="width:800px; display: inline;">

With the entire cascade of frames exhausted, a value is returned to ***frame A***, and is assigned to the variable `faces`.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.B06.jpg" style="width:800px; display: inline;">

All of this resulted from just one call to the `subdivide()` function, and produced a single value. This same choreography is repeated often over the course of a script. Any one of the expressions found in this code can be similarly drilled down, and are neither limited to the bounds of the script from which it is appears nor those of the imported module of the Decod.es library. The evaluation of a single script can make use of the entirety of the Python language, and the frames spawned by the procedures can well extend down to this lowest of levels. 

A textual account of the stack is given when an error occurs during a function call, at which time Python prints the name of every function in the entire stack, called a ***traceback***. This abbreviated historical account of execution can be used effectively to find the location and nature of the error: in what program file, in what line and what functions were executing at the time.